# Cosmological Power Spectra Calculation
This code calculates and stores cosmological power spectra 
and related coefficients for weak lensing and matter power spectra based on cosmological models.

In [1]:
import os
import json
import numpy
import pyccl
import scipy

In [ ]:
# FOLDER
FOLDER = '/global/cfs/cdirs/lsst/groups/MCP/CosmoCloud/LimberCloud/'
INFO_FOLDER = os.path.join(FOLDER, 'INFO')

Choosing a cosmological model.

In [ ]:
# Cosmology
with open(os.path.join(INFO_FOLDER, 'COSMOLOGY.json'), 'r') as file:
    COSMOLOGY_INFO = json.load(file)

COSMOLOGY = pyccl.Cosmology(
    h = COSMOLOGY_INFO['H'],
    w0 = COSMOLOGY_INFO['W0'],
    wa = COSMOLOGY_INFO['WA'], 
    n_s = COSMOLOGY_INFO['NS'], 
    A_s = COSMOLOGY_INFO['AS'],
    m_nu = COSMOLOGY_INFO['M_NU'],  
    Neff = COSMOLOGY_INFO['N_EFF'],
    Omega_b = COSMOLOGY_INFO['OMEGA_B'], 
    Omega_k = COSMOLOGY_INFO['OMEGA_K'], 
    Omega_c = COSMOLOGY_INFO['OMEGA_CDM'], 
    mass_split = 'single', matter_power_spectrum = 'halofit', transfer_function = 'boltzmann_camb',
    extra_parameters = {'camb': {'kmax': 50, 'lmax': 5000, 'halofit_version': 'mead2020_feedback', 'HMCode_logT_AGN': 7.8}}
)

Calculate the power spectrum.

In [ ]:
# Define the redshift grid
Z1 = 0.0
Z2 = 3.5
GRID_SIZE = 350
Z_GRID = numpy.linspace(Z1, Z2, GRID_SIZE + 1)

# Calculate corresponding scale factors and comoving distances
A_GRID = 1 / (1 + Z_GRID)
CHI_GRID = pyccl.background.comoving_radial_distance(cosmo=COSMOLOGY, a=A_GRID)

CHI_SIZE = 500
Z_DATA = numpy.linspace(Z1, Z2, CHI_SIZE + 1)

A_DATA = 1 / (1 + Z_DATA)
CHI_DATA = pyccl.background.comoving_radial_distance(cosmo=COSMOLOGY, a=A_DATA)

ELL1 = 20
ELL2 = 2000
ELL_SIZE = 20
ELL_GRID = numpy.geomspace(ELL1, ELL2, ELL_SIZE + 1)
CHI_MESH, ELL_MESH = numpy.meshgrid(CHI_GRID, ELL_GRID)
SCALE_GRID = numpy.nan_to_num(numpy.divide(ELL_MESH + 1/2, CHI_MESH, out=numpy.zeros((ELL_SIZE + 1, GRID_SIZE + 1)) + numpy.inf, where=CHI_MESH > 0))

# Calculate the power spectrum on the grid
POWER_GRID = numpy.zeros((ELL_SIZE + 1, GRID_SIZE + 1))
for GRID_INDEX in range(GRID_SIZE + 1):
    POWER_GRID[:,GRID_INDEX] = pyccl.power.nonlin_matter_power(cosmo=COSMOLOGY, k=SCALE_GRID[:,GRID_INDEX], a=A_GRID[GRID_INDEX])

Compute the numerical integral needed for the covariance matrix.

In [5]:
# Numerical integral

def integral_I2(chi1, chi2, chi3, power1, power2, redshift1, redshift2):
    
    def formula_0(chi):
        result = (chi2 ** 2 - chi ** 2) / 2 / (chi2 - chi1) - chi * chi2 * numpy.log(chi2 / chi) / (chi2 - chi1)
        
        result = result * ((chi3 - chi2) / 2 - chi * chi3 * numpy.log(chi3 / chi2) / (chi3 - chi2) + (chi2 ** 2 - 2 * chi1 * chi2 + chi ** 2) / 2 / (chi2 - chi1) + chi * chi1 * numpy.log(chi2 / chi) / (chi2 - chi1))
        
        result = result * ((chi / chi2) ** 3) * power2
        
        result = result * (1 + (chi2 - chi) / (chi2 - chi1) * redshift1 + (chi - chi1) / (chi2 - chi1) * redshift2) ** 2
        
        return result 
    
    def formula_n(chi):
        result = (chi2 ** 2 - chi ** 2) / 2 / (chi2 - chi1) - chi * chi2 * numpy.log(chi2 / chi) / (chi2 - chi1)
        
        result = result * ((chi3 - chi2) / 2 - chi * chi3 * numpy.log(chi3 / chi2) / (chi3 - chi2) + (chi2 ** 2 - 2 * chi1 * chi2 + chi ** 2) / 2 / (chi2 - chi1) + chi * chi1 * numpy.log(chi2 / chi) / (chi2 - chi1))
        
        result = result * ((chi2 - chi) / (chi2 - chi1) * power1 + (chi - chi1) / (chi2 - chi1) * power2) 
        
        result = result * (1 + (chi2 - chi) / (chi2 - chi1) * redshift1 + (chi - chi1) / (chi2 - chi1) * redshift2) ** 2
        
        return result
    
    if chi1 == 0:
        integral = scipy.integrate.quad_vec(f=formula_0, a=chi1, b=chi2)[0]
    else:
        integral = scipy.integrate.quad_vec(f=formula_n, a=chi1, b=chi2)[0]
    return integral

Calculate the coefficients that are used in the angular power spectrum computation.

In [6]:
# Coefficient

def coefficient_J2(chi1, chi2, chi3, power1, power2, redshift1, redshift2):
    a = 1 - chi1 / chi2
    b = chi3 / chi2 - 1
    p = 1 - power1 / power2
    z = 1 - (1 + redshift1) / (1 + redshift2)
    
    if a == 1:
        formula = (b * (6642 + z * ( - 6669 + 1867 * z) + 18 * b * (294 + z * ( - 308 + 89 * z))) - 45 * (1 + b) * (112 + z * ( - 104 + 27 * z)) * numpy.log(1 + b)) / (6350400 * b)
    else:
        formula = - ((1 / (4 * a ** 5 * b)) * ((1 / 15) * ( - 1 + a) ** 4 * b * (5 * a ** 2 * (p + a * ( - 4 + 3 * p)) - 2 * a * (2 * p + a * ( - 5 + 6 * p + 3 * a * ( - 5 + 4 * p))) * z + 
( - 2 * a * (1 + 3 * a + 6 * a ** 2) + p + a * (3 + 2 * a * (3 + 5 * a)) * p) * z ** 2) * numpy.log(1 - a) ** 2 + 
(1 / 3150) * (( - 1 + a) ** 2 * numpy.log(1 - a) * (b * (35 * a ** 2 * ( - 5 * a * (2 + a * (23 + a * ( - 52 + 9 * a) - 18 * b)) + (7 + a * (28 + a * (19 + 36 * ( - 5 + a) * a - 60 * b) - 30 * b)) * p) + 14 * a * (5 * a * (7 + a * (28 + a * (19 + 36 * ( - 5 + a) * a - 60 * b) - 30 * b)) - 26 * p + a * ( - 56 + 2 * a * ( - 13 + 3 * a * (4 + (114 - 25 * a) * a)) + 
75 * (1 + a * (2 + 3 * a)) * b) * p) * z + (7 * a * ( - 26 - 2 * a * (28 + a * (13 + 3 * a * ( - 4 + a * ( - 114 + 25 * a)))) + 75 * a * (1 + a * (2 + 3 * a)) * b) + 
(141 + a * (201 + 51 * a - 169 * a ** 2 - 424 * a ** 3 - 3850 * a ** 4 + 900 * a ** 5 - 315 * (1 + a * (2 + a * (3 + 4 * a))) * b)) * p) * z ** 2) - 
210 * ( - 1 + a) * a * (1 + b) * (5 * a ** 2 * (p + a * ( - 4 + 3 * p)) - 2 * a * (2 * p + a * ( - 5 + 6 * p + 3 * a * ( - 5 + 4 * p))) * z + 
( - 2 * a * (1 + 3 * a + 6 * a ** 2) + p + a * (3 + 2 * a * (3 + 5 * a)) * p) * z ** 2) * numpy.log(1 + b))) - 
(1 / 1323000) * (a * (b * (245 * a ** 2 * ( - 420 * p + 20 * a ** 2 * ( - 60 + 45 * b * ( - 6 + p) + 62 * p) + 9 * a ** 5 * ( - 195 + 148 * p) + 5 * a**3 * ( - 830 + 60 * b * (27 - 16 * p) + 193 * p) + 
150 * a * (4 + (5 + 12 * b) * p) + 2 * a ** 4 * (3725 - 2322 * p + 225 * b * ( - 4 + 3 * p))) - 
14 * a * (35 * a * (420 + a * ( - 150 * (5 + 12 * b) + a * ( - 20 * (62 + 45 * b) + a * ( - 965 + 4800 * b - 18 * a * ( - 258 + 74 * a + 75 * b))))) + 
( - 10920 + 2 * a * (9030 + a * (10360 + a * (7805 + 6 * a * (1008 + a * ( - 9688 + 3125 * a))))) + 525 * a * (60 + a * (30 + a * (20 + 9 * a * ( - 25 + 8 * a)))) * b) * p) * z + (7 * a * (10920 - 2 * a * (9030 + a * (10360 + a * (7805 + 6 * a * (1008 + a * ( - 9688 + 3125 * a))))) - 
525 * a * (60 + a * (30 + a * (20 + 9 * a * ( - 25 + 8 * a)))) * b) + ( - 59220 + a * (92610 + 85470 * a + 62685 * a ** 2 + 48111 * a ** 3 + 38584 * a ** 4 - 629500 * a ** 5 + 219375 * a**6 + 2205 * (60 + a * (30 + a * (20 + a * (15 + 4 * a * ( - 72 + 25 * a))))) * b)) * p) * z**2) + 
210 * a * (1 + b) * ( - 210 * a * (p * ( - 8 + z) - 4 * z) * z - 420 * p * z ** 2 - 35 * a ** 3 * (30 * ( - 8 + p) + 4 * (15 - 4 * p) * z + ( - 8 + 3 * p) * z ** 2) + 
14 * a ** 4 * ( - 1500 + 850 * p + 5 * (340 - 117 * z) * z + 6 * p * z * ( - 195 + 74 * z)) - 140 * a ** 2 * ( - 3 * ( - 10 + z) * z + p * (15 + ( - 6 + z) * z)) + 
30 * a ** 6 * (4 * p * (21 + 5 * z * ( - 7 + 3 * z)) - 7 * (15 + 2 * z * ( - 12 + 5 * z))) - 7 * a**5 * ( - 2200 + 18 * (175 - 68 * z) * z + p * (1575 + 8 * z * ( - 306 + 125 * z)))) * numpy.log(1 + b)))))
    
    coefficient = chi2 ** 3 * power2 * (1 + redshift2) ** 2 * formula
    return coefficient

In [7]:
# Case1: chi1 > 0

ELL_INDEX = 0
GRID_INDEX = int(GRID_SIZE / 2)

CHI1 = numpy.float128(CHI_GRID[GRID_INDEX])
CHI2 = numpy.float128(CHI_GRID[GRID_INDEX + 1])
CHI3 = numpy.float128(CHI_GRID[GRID_INDEX + 2])

POWER1 = numpy.float128(POWER_GRID[ELL_INDEX,GRID_INDEX])
POWER2 = numpy.float128(POWER_GRID[ELL_INDEX,GRID_INDEX + 1])

REDSHIFT1 = numpy.float128(Z_GRID[GRID_INDEX])
REDSHIFT2 = numpy.float128(Z_GRID[GRID_INDEX + 1])

print(CHI1, CHI2, CHI3, POWER1, POWER2, REDSHIFT1, REDSHIFT2)

4924.221744025123371 4940.716645699534638 4957.1305889213954288 11370.147631619063759 11268.969970366384587 1.75 1.7600000000000000089


Compute the integral and coefficient for the case where chi1 > 0

In [8]:
# Case1: chi1 > 0

INTEGRAL = integral_I2(CHI1, CHI2, CHI3, POWER1, POWER2, REDSHIFT1, REDSHIFT2)

COEFFICIENT = coefficient_J2(CHI1, CHI2, CHI3, POWER1, POWER2, REDSHIFT1, REDSHIFT2)

print(INTEGRAL, COEFFICIENT, COEFFICIENT / INTEGRAL - 1)

143.64987918366898305 144.28632661235668473 0.004430546216289870721


In [9]:
# Case2: chi1 = 0

ELL_INDEX = 0
GRID_INDEX = 0

CHI1 = CHI_GRID[GRID_INDEX]
CHI2 = CHI_GRID[GRID_INDEX + 1]
CHI3 = CHI_GRID[GRID_INDEX + 2]

POWER1 = POWER_GRID[ELL_INDEX,GRID_INDEX]
POWER2 = POWER_GRID[ELL_INDEX,GRID_INDEX + 1]

REDSHIFT1 = Z_GRID[GRID_INDEX]
REDSHIFT2 = Z_GRID[GRID_INDEX + 1]

print(CHI1, CHI2, CHI3, POWER1, POWER2, REDSHIFT1, REDSHIFT2)

0.0 44.40057919571794 88.58943766284152 0.0 1826.8838600584127 0.0 0.01


Compute the integral and coefficient for the case where chi1 = 0

In [10]:
# Case2: chi1 = 0

INTEGRAL = integral_I2(CHI1, CHI2, CHI3, POWER1, POWER2, REDSHIFT1, REDSHIFT2)

COEFFICIENT = coefficient_J2(CHI1, CHI2, CHI3, POWER1, POWER2, REDSHIFT1, REDSHIFT2)

print(INTEGRAL, COEFFICIENT, COEFFICIENT / INTEGRAL - 1)

125171.83112209634 125171.83112211828 1.7541523789077473e-13
